In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

account = Account.get("Ed")
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "ed", "balance": 8720.28568, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.6264, "timestamp": "2025-06-12 11:40:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 212.94504, "timestamp": "2025-06-19 18:44:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-12 11:40:42", 9998.720800000001], ["2025-06-12 11:40:56", 9998.720800000001], ["2025-06-19 18:44:31", 9995.405680000002]], "total_portfolio_value": 9995.405680000002, "total_profit_loss": -4.5943200000001525}'

In [3]:
account.report()

'{"name": "ed", "balance": 8720.28568, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.6264, "timestamp": "2025-06-12 11:40:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 212.94504, "timestamp": "2025-06-19 18:44:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-12 11:40:42", 9998.720800000001], ["2025-06-12 11:40:56", 9998.720800000001], ["2025-06-19 18:44:31", 9995.405680000002], ["2025-06-19 18:44:37", 9995.405680000002]], "total_portfolio_value": 9995.405680000002, "total_profit_loss": -4.5943200000001525}'

In [4]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 213.6264,
  'timestamp': '2025-06-12 11:40:42',
  'rationale': 'Because this bookstore website looks promising'},
 {'symbol': 'AMZN',
  'quantity': 3,
  'price': 212.94504,
  'timestamp': '2025-06-19 18:44:31',
  'rationale': 'Because this bookstore website looks promising'}]

In [5]:
params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='get_balance', description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, annotations=None),
 Tool(name='get_holdings', description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, annotations=None),
 Tool(name='buy_shares', description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rationale for the purchase and fit with the account's strategy\n    ", inputSchema={'properties': {'name': {'title': 'Name', '

In [6]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Ed and my account is under the name Ed. What's my balance and my holdings?"
model = "gpt-4.1-mini"

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Ed, your current cash balance is approximately $8,720.29. You hold 6 shares of Amazon (AMZN) in your account. If you need any more details or want to make transactions, just let me know!

Now Mcp Client Will be used


In [9]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools


openai_tools = await get_accounts_tools_openai()
with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

context = await read_accounts_resource("ed")
print(context)

Ed, your current cash balance is approximately $8,720.29. In terms of holdings, you have 6 shares of Amazon (AMZN). Is there anything specific you would like to do with your account?

{"name": "ed", "balance": 8720.28568, "strategy": "", "holdings": {"AMZN": 6}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 213.6264, "timestamp": "2025-06-12 11:40:42", "rationale": "Because this bookstore website looks promising"}, {"symbol": "AMZN", "quantity": 3, "price": 212.94504, "timestamp": "2025-06-19 18:44:31", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-06-12 11:40:42", 9998.720800000001], ["2025-06-12 11:40:56", 9998.720800000001], ["2025-06-19 18:44:31", 9995.405680000002], ["2025-06-19 18:44:37", 9995.405680000002], ["2025-06-19 18:50:32", 9995.405680000002]], "total_portfolio_value": 9995.405680000002, "total_profit_loss": -4.5943200000001525}
